In [1]:
#import matplotlib.pyplot as plt
#from osgeo import gdal # Import the GDAL library
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import requests
import cdsapi

In [7]:
c = cdsapi.Client()
time = ['00:00', '01:00', '02:00','03:00', '04:00', '05:00',
        '06:00', '07:00', '08:00','09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00','15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00','21:00', '22:00', '23:00',]
days = ['01','02','03','04','05'
        ]        #,'03','04','05','06','07','08','09','10','11','12','13','14''18','19','20','21','22','23','24','25','26','27','28','29','30'
        
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure',
        ],
        'year': '2015',
        'month': '10',
        'day': days,

        'time': time,
    },
    'joaquin2.grib')

2020-04-06 18:58:03,066 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-04-06 18:58:03,717 INFO Request is queued
2020-04-06 18:58:04,815 INFO Request is running
2020-04-06 18:58:25,077 INFO Request is completed
2020-04-06 18:58:25,078 INFO Downloading http://136.156.132.236/cache-compute-0007/cache/data5/adaptor.mars.internal-1586213885.097692-5149-39-4489739b-4d95-4986-bfce-921a9ed92091.grib to joaquin2.grib (712.9M)
2020-04-06 18:59:58,566 INFO Download rate 7.6M/s                                                                      


Result(content_length=747576000,content_type=application/x-grib,location=http://136.156.132.236/cache-compute-0007/cache/data5/adaptor.mars.internal-1586213885.097692-5149-39-4489739b-4d95-4986-bfce-921a9ed92091.grib)

In [8]:
path = pl.Path(r'D:\FHRL_work\libraries\adcirc-unswan\inputs')
file = 'joaquin2.grib'
grib = xr.open_dataset(path  / file, engine='cfgrib')

2020-04-06 19:00:00,025 INFO missing from GRIB stream: 'directionNumber'
2020-04-06 19:00:00,026 INFO missing from GRIB stream: 'frequencyNumber'
2020-04-06 19:00:00,414 INFO missing from GRIB stream: 'directionNumber'
2020-04-06 19:00:00,415 INFO missing from GRIB stream: 'frequencyNumber'
2020-04-06 19:00:00,720 INFO missing from GRIB stream: 'directionNumber'
2020-04-06 19:00:00,720 INFO missing from GRIB stream: 'frequencyNumber'


In [2]:
path = pl.Path(r'D:\temp')
file = 'wnd10m.gdas.201610.grib2'
grib = xr.open_dataset(path  / file, engine='cfgrib')

In [3]:
grib

<xarray.Dataset>
Dimensions:            (latitude: 880, longitude: 1760, step: 7, time: 124)
Coordinates:
  * time               (time) datetime64[ns] 2016-10-01 ... 2016-10-31T18:00:00
  * step               (step) timedelta64[ns] 00:00:00 01:00:00 ... 06:00:00
    heightAboveGround  int32 ...
  * latitude           (latitude) float64 89.84 89.64 89.44 ... -89.64 -89.84
  * longitude          (longitude) float64 0.0 0.2045 0.4091 ... 359.6 359.8
    valid_time         (time, step) datetime64[ns] ...
Data variables:
    u10                (time, step, latitude, longitude) float32 ...
    v10                (time, step, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    history:                 2020-04-09T13:19:36 GRIB to CDM+CF via cfgrib-0....

In [4]:
#u = grib.data_vars['u10'].values[:,:,:,:]
#v = grib.data_vars['v10'].values[:,:,:,:]


In [14]:
time = grib.coords['time'].values[:]
step = grib.coords['step'].values[:]
lat = grib.coords['latitude'].values
lon = grib.coords['longitude'].values

# if you want to split the time
start = '2016-10-05 00:00:00'
end = '2016-10-20 00:00:00'
start_split = np.where(((pd.to_datetime(start)-pd.Timedelta(hours=1))<pd.to_datetime(time))&(pd.to_datetime(time)<=(pd.to_datetime(start)+pd.Timedelta(hours=1))))

In [15]:
start_split

(array([16], dtype=int64),)

In [12]:
time.

ValueError: cannot include dtype 'M' in a buffer

In [7]:
start_split

(array([], dtype=int64),)

## For CFSv2 monthly reanylisis 

In [32]:
path = pl.Path(r'D:\temp')
file = 'prmsl.gdas.201610.grib2'
grib = xr.open_dataset(path / file , engine='cfgrib')
prmsl = grib.variables['prmsl'].values[:,:,:,:]

lat1,lat2 = -80,20
lon1,lon2 = -82, -0.5

# ---- generate adcirc forcing file (fort.22)---------

t1,t2,y,x = u.shape
u2,v2,p2 = [],[],[]
datay,datax = [], []
for t in range(t1):
    for tt in range(0,t2,1):
        for i in range(0,y):
            for ii in range(0,x):
                if lat2>=lat[i]>lat1 and (lon1+360)<=lon[ii]<(lon2+360):
                    datay.append(i)
                    datax.append(ii)
                    u2.append(np.round(u[t,tt,i,ii],3))
                    v2.append(np.round(v[t,tt,i,ii],3))
                    y2 = int(np.floor(i*0.409))
                    x2 = int(np.floor(ii*0.409))
                    p2.append(prmsl[t,tt,y2,x2])
    print(len(u2),len(v2),len(p2))
df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('fort.22', sep='\t',index=False,header = False)     


1362354 1362354 1362354
2724708 2724708 2724708
4087062 4087062 4087062
5449416 5449416 5449416
6811770 6811770 6811770
8174124 8174124 8174124
9536478 9536478 9536478
10898832 10898832 10898832
12261186 12261186 12261186
13623540 13623540 13623540
14985894 14985894 14985894
16348248 16348248 16348248
17710602 17710602 17710602
19072956 19072956 19072956
20435310 20435310 20435310
21797664 21797664 21797664
23160018 23160018 23160018
24522372 24522372 24522372
25884726 25884726 25884726
27247080 27247080 27247080


In [33]:
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(489, 398)

## For ERA 5

In [ ]:
lat1,lat2 = -80,20
lon1,lon2 = -82, -0.5

lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = np.arange(0,len(grib.coords['time'][:]))

In [11]:
lat1,lat2 = 7,60
lon1,lon2 = -100, -58

lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = np.arange(0,len(grib.coords['time'][:]))


temp,y,x = u.shape
u2,v2,p2 = [],[],[]
datay,datax = [], []
for t in time:
    for i in range(0,y):
        for ii in range(0,x):
            if (lat2>=lat[i]>lat1) and ((lon2+360)>=lon[ii]>(lon1+360)):
                u2.append(np.round(u[t,i,ii],3))
                v2.append(np.round(v[t,i,ii],3))
                p2.append(prmsl[t,i,ii])
                datay.append(i)
                datax.append(ii)

df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('joaquin2.22', sep='\t',index=False,header = False)  

temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

## Potentially GFS and GEFS

In [3]:
url = 'ftp://nomads.ncdc.noaa.gov/GFS/analysis_only/200807/20080701/'

day = 1
data = {}
u_wind,v_wind,psl = [],[],[]
name = []
for m in range(7,8):
    for t in range(1,2):
        if t < 10:
            t_s = '0'+str(t)
        else:
            t_s = str(t)
        url =  'ftp://nomads.ncdc.noaa.gov/GFS/analysis_only/20080{}/20080{}{}/'.format(str(m),str(m),t_s)

        for hour in range(0,1):
            if hour==0 or hour==6:
                time = '0'+str(hour)
            else:
                time = str(hour)
            
            file = 'gfsanl_3_20080{}{}_{}00_000.grb'.format(m,t_s,time)
            print(file)
            name.append(file)
            test = gdal.Open(url+file)
'''
            for key in dataset.keys():
                temp = test.GetRasterBand(dataset[key])
                data[key] = temp.ReadAsArray()

            for i in range(0,90):
                for ii in range(160,245):
                    u_wind.append(round(data['u_wind'][i,ii],3))
                    v_wind.append(round(data['v_wind'][i,ii],3))
                    psl.append(round(data['psl'][i,ii],3))
        if t==20 and m==8:
            break
    
df = pd.DataFrame({'u':u_wind,'v':v_wind,'p':psl})
df.to_csv('fort.22', sep='\t',index=False,header = False)
'''

gfsanl_3_20080701_0000_000.grb


"\n            for key in dataset.keys():\n                temp = test.GetRasterBand(dataset[key])\n                data[key] = temp.ReadAsArray()\n\n            for i in range(0,90):\n                for ii in range(160,245):\n                    u_wind.append(round(data['u_wind'][i,ii],3))\n                    v_wind.append(round(data['v_wind'][i,ii],3))\n                    psl.append(round(data['psl'][i,ii],3))\n        if t==20 and m==8:\n            break\n    \ndf = pd.DataFrame({'u':u_wind,'v':v_wind,'p':psl})\ndf.to_csv('fort.22', sep='\t',index=False,header = False)\n"